In [ ]:
import polars as pl
import validate.lesion as vl
import validate.general as vg

from constants import list_id_cols

import utils
from pathlib import Path
import os

PATH_INPUT = os.getenv("PATH_INPUT")

lf = utils.get_df(Path(PATH_INPUT).joinpath("test data.accdb")).lazy()

lf2 = utils.get_df(Path(PATH_INPUT).joinpath("ZON CHERAS OC DATABASE JULY-OCT 2023 edit MAIwani.accdb")).lazy()

obj = vg.ValidationGeneral(lf, "test data")
lfall = pl.concat([lf, lf2])

In [ ]:
lfall.columns

In [ ]:
print(
  lfall.select("HADIR QUIT SERVICES", "STATUS INTERVENSI").group_by("*").agg().collect()
)


In [ ]:
from datetime import date

today = date.today()
today

In [ ]:
lfall.select("TARIKH TEMUJANJI QUIT SERVICE", "HADIR QUIT SERVICES").filter(
  (pl.col("TARIKH TEMUJANJI QUIT SERVICE") < today)
  & pl.col("HADIR QUIT SERVICES").is_null()
).collect()
